extractcsv.py

In [25]:
import pandas as pd

def read_csvfile_into_dataframe(file_name: str):
    try:
        df = pd.read_csv(file_name)     
        return df
    except FileNotFoundError:
        return None

In [26]:
read_csvfile_into_dataframe('test.csv')

,timestamp,store,customer_name,basket_items,total_price,cash_or_card,card_number
0,06/06/2022 09:00,Chesterfield,Stephanie Neyhart,"Large Flat white - 2.45, Large Flavoured iced ...",8.95,CASH,NaN
1,06/06/2022 09:02,Chesterfield,Donna Marley,"Large Flavoured iced latte - Hazelnut - 3.25, ...",8.65,CARD,4.694130e+15
2,06/06/2022 09:04,Chesterfield,Frances Pounds,"Regular Flavoured iced latte - Caramel - 2.75,...",8.15,CASH,NaN
3,06/06/2022 09:06,Chesterfield,Veronica Ikenberry,Large Flat white - 2.45,2.45,CASH,NaN
4,06/06/2022 09:08,Chesterfield,Rachelle Blandy,"Large Flavoured latte - Hazelnut - 2.85, Large...",13.55,CARD,6.577560e+15
...,...,...,...,...,...,...,...
222,06/06/2022 16:53,Chesterfield,Daniela Baptiste,"Large Flat white - 2.45, Large Flat white - 2....",12.65,CASH,NaN
223,06/06/2022 16:55,Chesterfield,Christopher Duquette,"Large Flavoured iced latte - Hazelnut - 3.25, ...",10.30,CARD,9.596730e+15
224,06/06/2022 16:57,Chesterfield,Geraldine Sykes,Regular Flavoured iced latte - Hazelnut - 2.75...,10.10,CARD,5.925480e+15
225,06/06/2022 16:57,Chesterfield,Geraldine Sykes,"Regular Flavoured Chai latte - Vanilla - 2.75,...",10.10,CARD,5.925480e+15


transform_3nf.py

In [27]:
import pandas as pd
import numpy as np
# from create_db import connect_t_db

#from product_query import query_id
TEST_CSV = 'test.csv' # Located in 'src' folder
BASKET_COLUMN = 'basket_items'

# Columns to be extracted for products_df, index column acts as a local primary key
PRODUCT_COLUMNS = ["name", "size", "flavour"]

# Columns to be extracted for transactions_df, after UUIDs have been generated
TRANSACTION_COLUMNS = ["timestamp", "store", "customer_name", "total_price", "cash_or_card"]

# TRANSACTION_COLUMNS = ["transaction_id","timestamp", "store", "customer_name", "total_price", "cash_or_card"]
TRANSACTION_ID = "transaction_id"
PRODUCT_ID = "product_id"
BASKET_ITEMS_COLUMNS = [TRANSACTION_ID, PRODUCT_ID, "price", "quantity"]

# BASKET_ITEMS_COLUMNS = ["transaction_id", "product_id", "price", "quantity","timestamp", "store", "customer_name", "total_price", "cash_or_card","basket_items"]
TRANSACTIONS_TABLE = "transactions"
PRODUCTS_TABLE = "products"
BASKET_ITEMS_TABLE = "basket_items"

In [28]:
# Transform data_frame to 1st Normal Form
# We will split the basket items into separate rows after each comma (counting quantity per basket)
def split_basket_items(df:pd.DataFrame):
    
    def count_items(basket: str):

        # empty dictionary
        item_count_dict = {}

        # list of items in basket stripped and split by ','
        items_list = [item.strip() for item in basket.split(",")]

        for item in items_list:

            # 'get' returns key value pair for dict
            item_count_dict[item] = item_count_dict.get(item, 0) + 1

        # 
        return list(item_count_dict.items())

    # 
    df[BASKET_COLUMN] = df[BASKET_COLUMN].apply(lambda x: count_items(x))
    
    # "explode" converts each element into row
    return df.explode(BASKET_COLUMN, ignore_index=True)

In [29]:
split_basket_items(read_csvfile_into_dataframe(TEST_CSV))

,timestamp,store,customer_name,basket_items,total_price,cash_or_card,card_number
0,06/06/2022 09:00,Chesterfield,Stephanie Neyhart,"(Large Flat white - 2.45, 1)",8.95,CASH,NaN
1,06/06/2022 09:00,Chesterfield,Stephanie Neyhart,"(Large Flavoured iced latte - Vanilla - 3.25, 1)",8.95,CASH,NaN
2,06/06/2022 09:00,Chesterfield,Stephanie Neyhart,"(Large Flavoured iced latte - Hazelnut - 3.25, 1)",8.95,CASH,NaN
3,06/06/2022 09:02,Chesterfield,Donna Marley,"(Large Flavoured iced latte - Hazelnut - 3.25, 1)",8.65,CARD,4.694130e+15
4,06/06/2022 09:02,Chesterfield,Donna Marley,"(Regular Latte - 2.15, 1)",8.65,CARD,4.694130e+15
...,...,...,...,...,...,...,...
569,06/06/2022 16:57,Chesterfield,Geraldine Sykes,(Regular Flavoured iced latte - Vanilla - 2.75...,10.10,CARD,5.925480e+15
570,26/06/2022 16:57,Chesterfield,Geraldine Smith,(Regular Flavoured Chai latte - Vanilla - 2.75...,10.10,CARD,5.925480e+15
571,26/06/2022 16:57,Chesterfield,Geraldine Smith,"(Large Latte - 2.45, 1)",10.10,CARD,5.925480e+15
572,26/06/2022 16:57,Chesterfield,Geraldine Smith,"(Regular Flat white - 2.15, 1)",10.10,CARD,5.925480e+15


In [30]:
# Split the product column into separate columns for each detail: size, name, flavour, price, quantity
def extract_product_details(df:pd.DataFrame):

    # Extract the details of product tuple ((size, name, flavour, price), quantity) into a list (size, name, flavour, price, quantity)
    def extract_details(product_count:tuple):
        
        # select first element of tuple: size, name, flavour, price (and exclude quantity)
        product = product_count[0]

        # split size from name, flavour, price
        size_plus_other_details = product.strip().split(" ", 1)

        # size (first element of tuple)
        size = size_plus_other_details[0]

        # split (by "-") name, flavour, price (2nd element of tuple)
        other_details = [detail.strip() for detail in size_plus_other_details[1].split("-")] 
        
        # if 2 elements then flavour = 'original'
        if len(other_details) == 2:

            # price is last element
            price = other_details[-1]

            # define flavour as 'original'
            other_details[-1] = "Original"
            
            # append price
            other_details.append(price)
        
        # name is 1st element
        name = other_details[0]

        # 
        if name[:9] == "Flavoured":
            other_details[0] = name[9:].lstrip().capitalize()
        
        details = [size]
        details.extend(other_details)
        details.append(product_count[1])

        return details
    
    df[BASKET_COLUMN] = df[BASKET_COLUMN].apply(extract_details)
    
    df[['size','name','flavour','price','quantity']] = pd.DataFrame(df[BASKET_COLUMN].to_list())
    
    del df[BASKET_COLUMN]
    
    return df

In [31]:
extract_product_details(split_basket_items(read_csvfile_into_dataframe(TEST_CSV)))

,timestamp,store,customer_name,total_price,cash_or_card,card_number,size,name,flavour,price,quantity
0,06/06/2022 09:00,Chesterfield,Stephanie Neyhart,8.95,CASH,NaN,Large,Flat white,Original,2.45,1
1,06/06/2022 09:00,Chesterfield,Stephanie Neyhart,8.95,CASH,NaN,Large,Iced latte,Vanilla,3.25,1
2,06/06/2022 09:00,Chesterfield,Stephanie Neyhart,8.95,CASH,NaN,Large,Iced latte,Hazelnut,3.25,1
3,06/06/2022 09:02,Chesterfield,Donna Marley,8.65,CARD,4.694130e+15,Large,Iced latte,Hazelnut,3.25,1
4,06/06/2022 09:02,Chesterfield,Donna Marley,8.65,CARD,4.694130e+15,Regular,Latte,Original,2.15,1
...,...,...,...,...,...,...,...,...,...,...,...
569,06/06/2022 16:57,Chesterfield,Geraldine Sykes,10.10,CARD,5.925480e+15,Regular,Iced latte,Vanilla,2.75,1
570,26/06/2022 16:57,Chesterfield,Geraldine Smith,10.10,CARD,5.925480e+15,Regular,Chai latte,Vanilla,2.75,1
571,26/06/2022 16:57,Chesterfield,Geraldine Smith,10.10,CARD,5.925480e+15,Large,Latte,Original,2.45,1
572,26/06/2022 16:57,Chesterfield,Geraldine Smith,10.10,CARD,5.925480e+15,Regular,Flat white,Original,2.15,1


In [32]:
# Extract some columns for a subtable: 'transactions' and 'products', and replace with row_index
def extract_subtable(df:pd.DataFrame, subcolumns:list, foreign_key:str):
    subtable_df = df[subcolumns].drop_duplicates()

    subtable_df['combined'] = subtable_df.astype(str).apply(np.sum, axis=1)
    
    def get_index(combined_value:str, subtable_df:pd.DataFrame):
        try:
            return subtable_df.index[subtable_df['combined'] == combined_value].tolist()[0]
        except:
            return None

    df[foreign_key] = df[subcolumns].astype(str).apply(np.sum, axis=1).apply(lambda x: get_index(x, subtable_df))
    del subtable_df['combined']

    for column in subcolumns:
        del df[column]
    
    return subtable_df

In [33]:
extract_subtable(read_csvfile_into_dataframe(TEST_CSV),TRANSACTION_COLUMNS,TRANSACTION_ID)

,timestamp,store,customer_name,total_price,cash_or_card
0,06/06/2022 09:00,Chesterfield,Stephanie Neyhart,8.95,CASH
1,06/06/2022 09:02,Chesterfield,Donna Marley,8.65,CARD
2,06/06/2022 09:04,Chesterfield,Frances Pounds,8.15,CASH
3,06/06/2022 09:06,Chesterfield,Veronica Ikenberry,2.45,CASH
4,06/06/2022 09:08,Chesterfield,Rachelle Blandy,13.55,CARD
...,...,...,...,...,...
221,06/06/2022 16:51,Chesterfield,Jack Livengood,4.90,CASH
222,06/06/2022 16:53,Chesterfield,Daniela Baptiste,12.65,CASH
223,06/06/2022 16:55,Chesterfield,Christopher Duquette,10.30,CARD
224,06/06/2022 16:57,Chesterfield,Geraldine Sykes,10.10,CARD


In [34]:
extract_subtable(extract_product_details(split_basket_items(read_csvfile_into_dataframe(TEST_CSV))), PRODUCT_COLUMNS, PRODUCT_ID)

,name,size,flavour
0,Flat white,Large,Original
1,Iced latte,Large,Vanilla
2,Iced latte,Large,Hazelnut
4,Latte,Regular,Original
6,Iced latte,Regular,Caramel
7,Latte,Large,Hazelnut
8,Latte,Regular,Hazelnut
12,Iced latte,Regular,Vanilla
15,Iced latte,Large,Caramel
18,Latte,Large,Original


In [35]:
def third_normal_form(df: pd.DataFrame):
    
    # transactions table
    transactions_df = extract_subtable(df, TRANSACTION_COLUMNS, TRANSACTION_ID)

    # products table
    basket_df = split_basket_items(df)
    extract_product_details(basket_df)
    products_df = extract_subtable(basket_df, PRODUCT_COLUMNS, PRODUCT_ID)
    
    return {
        PRODUCTS_TABLE: products_df,
        TRANSACTIONS_TABLE: transactions_df,
        BASKET_ITEMS_TABLE: basket_df[BASKET_ITEMS_COLUMNS]
    }

In [36]:
third_normal_form(read_csvfile_into_dataframe(TEST_CSV))

{'products':            name     size   flavour
 0    Flat white    Large  Original
 1    Iced latte    Large   Vanilla
 2    Iced latte    Large  Hazelnut
 4         Latte  Regular  Original
 6    Iced latte  Regular   Caramel
 7         Latte    Large  Hazelnut
 8         Latte  Regular  Hazelnut
 12   Iced latte  Regular   Vanilla
 15   Iced latte    Large   Caramel
 18        Latte    Large  Original
 20   Iced latte  Regular  Hazelnut
 35   Flat white  Regular  Original
 566  Chai latte  Regular   Vanilla,
 'transactions':             timestamp         store         customer_name  total_price  \
 0    06/06/2022 09:00  Chesterfield     Stephanie Neyhart         8.95   
 1    06/06/2022 09:02  Chesterfield          Donna Marley         8.65   
 2    06/06/2022 09:04  Chesterfield        Frances Pounds         8.15   
 3    06/06/2022 09:06  Chesterfield    Veronica Ikenberry         2.45   
 4    06/06/2022 09:08  Chesterfield       Rachelle Blandy        13.55   
 ..              

In [37]:
third_normal_form(read_csvfile_into_dataframe(TEST_CSV))['products'].to_csv('products.csv')

In [38]:
print('PRODUCTS DATAFRAME')
third_normal_form(read_csvfile_into_dataframe(TEST_CSV))['products']

PRODUCTS DATAFRAME


,name,size,flavour
0,Flat white,Large,Original
1,Iced latte,Large,Vanilla
2,Iced latte,Large,Hazelnut
4,Latte,Regular,Original
6,Iced latte,Regular,Caramel
7,Latte,Large,Hazelnut
8,Latte,Regular,Hazelnut
12,Iced latte,Regular,Vanilla
15,Iced latte,Large,Caramel
18,Latte,Large,Original


In [39]:
third_normal_form(read_csvfile_into_dataframe(TEST_CSV))['products'].to_string()

'           name     size   flavour\n0    Flat white    Large  Original\n1    Iced latte    Large   Vanilla\n2    Iced latte    Large  Hazelnut\n4         Latte  Regular  Original\n6    Iced latte  Regular   Caramel\n7         Latte    Large  Hazelnut\n8         Latte  Regular  Hazelnut\n12   Iced latte  Regular   Vanilla\n15   Iced latte    Large   Caramel\n18        Latte    Large  Original\n20   Iced latte  Regular  Hazelnut\n35   Flat white  Regular  Original\n566  Chai latte  Regular   Vanilla'

In [40]:
third_normal_form(read_csvfile_into_dataframe(TEST_CSV))['transactions'].to_csv('transactions.csv')

In [41]:
print('TRANSACTIONS DATAFRAME')
third_normal_form(read_csvfile_into_dataframe(TEST_CSV))['transactions']

TRANSACTIONS DATAFRAME


,timestamp,store,customer_name,total_price,cash_or_card
0,06/06/2022 09:00,Chesterfield,Stephanie Neyhart,8.95,CASH
1,06/06/2022 09:02,Chesterfield,Donna Marley,8.65,CARD
2,06/06/2022 09:04,Chesterfield,Frances Pounds,8.15,CASH
3,06/06/2022 09:06,Chesterfield,Veronica Ikenberry,2.45,CASH
4,06/06/2022 09:08,Chesterfield,Rachelle Blandy,13.55,CARD
...,...,...,...,...,...
221,06/06/2022 16:51,Chesterfield,Jack Livengood,4.90,CASH
222,06/06/2022 16:53,Chesterfield,Daniela Baptiste,12.65,CASH
223,06/06/2022 16:55,Chesterfield,Christopher Duquette,10.30,CARD
224,06/06/2022 16:57,Chesterfield,Geraldine Sykes,10.10,CARD


In [45]:
a = third_normal_form(read_csvfile_into_dataframe(TEST_CSV))['basket_items'].to_csv()
type(a)

str

In [20]:
print(third_normal_form(read_csvfile_into_dataframe(TEST_CSV))['basket_items'].to_csv())

,transaction_id,product_id,price,quantity
0,0,0,2.45,1
1,0,1,3.25,1
2,0,2,3.25,1
3,1,2,3.25,1
4,1,4,2.15,1
5,1,1,3.25,1
6,2,6,2.75,1
7,2,7,2.85,1
8,2,8,2.55,1
9,3,0,2.45,1
10,4,7,2.85,1
11,4,2,3.25,1
12,4,12,2.75,1
13,4,4,2.15,1
14,4,8,2.55,1
15,5,15,3.25,1
16,5,0,2.45,1
17,6,4,2.15,1
18,6,18,2.45,1
19,7,0,2.45,1
20,7,20,2.75,1
21,8,4,2.15,1
22,8,1,3.25,1
23,9,6,2.75,1
24,10,4,2.15,1
25,10,12,2.75,1
26,10,2,3.25,1
27,11,0,2.45,1
28,12,12,2.75,1
29,13,20,2.75,1
30,13,2,3.25,1
31,13,18,2.45,1
32,13,8,2.55,1
33,14,6,2.75,1
34,15,4,2.15,1
35,16,35,2.15,1
36,16,12,2.75,1
37,16,8,2.55,1
38,17,6,2.75,1
39,17,20,2.75,1
40,17,18,2.45,1
41,17,7,2.85,1
42,17,15,3.25,1
43,18,35,2.15,1
44,18,4,2.15,1
45,19,12,2.75,1
46,19,1,3.25,1
47,20,1,3.25,1
48,20,12,2.75,1
49,21,35,2.15,1
50,22,0,2.45,1
51,23,4,2.15,1
52,23,6,2.75,1
53,24,15,3.25,1
54,25,2,3.25,1
55,25,20,2.75,1
56,25,1,3.25,1
57,25,8,2.55,1
58,26,1,3.25,1
59,26,18,2.45,1
60,27,12,2.75,1
61,27,20,2.75,1
62,27,15,3.25,1
63,28,18,2.45,1
64,28,15

In [15]:
print('BASKET DATAFRAME')
third_normal_form(read_csvfile_into_dataframe(TEST_CSV))['basket_items']

BASKET DATAFRAME


,transaction_id,product_id,price,quantity
0,0,0,2.45,1
1,0,1,3.25,1
2,0,2,3.25,1
3,1,2,3.25,1
4,1,4,2.15,1
...,...,...,...,...
569,224,12,2.75,1
570,226,566,2.75,1
571,226,18,2.45,1
572,226,35,2.15,1
